In [1]:
from sklearn.grid_search import GridSearchCV
import kagglegym
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from time import time
import itertools

C:\Users\jiguo\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\jiguo\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
env = kagglegym.make()
o = env.reset()
#o.train = o.train[:1000]
excl = [env.ID_COL_NAME, env.SAMPLE_COL_NAME, env.TARGET_COL_NAME, env.TIME_COL_NAME]
col = [c for c in o.train.columns if c not in excl]



In [5]:
o.train.y.describe()

count    806298.000000
mean          0.000256
std           0.021847
min          -0.086094
25%          -0.009334
50%          -0.000007
75%           0.009511
max           0.093498
Name: y, dtype: float64

In [6]:
#train = pd.read_hdf(r'C:\Users\jiguo\Desktop\KProject\input\train.h5')
train = pd.read_hdf('../input/train.h5')
train = train[col]
d_mean= train.median(axis=0)



In [4]:
#only for trees
train = o.train[col]
n = train.isnull().sum(axis=1)
for c in train.columns:
    train[c + '_nan_'] = pd.isnull(train[c])
    d_mean[c + '_nan_'] = 0
train = train.fillna(d_mean)
train['znull'] = n
n = []



C:\Users\jiguo\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
# #https://www.kaggle.com/bguberfain/two-sigma-financial-modeling/univariate-model-with-clip/run/482189
low_y_cut = -0.075
high_y_cut = 0.075

y_is_above_cut = (o.train.y > high_y_cut)
y_is_below_cut = (o.train.y < low_y_cut)
y_is_within_cut = (~y_is_above_cut & ~y_is_below_cut)
model2 = LinearRegression(n_jobs=-1)
model2.fit(np.array(o.train[col].fillna(d_mean).loc[y_is_within_cut, 'technical_20'].values).reshape(-1,1), o.train.loc[y_is_within_cut, 'y'])


class LR_tech_20():
    def __init__(self, n_jobs=3):
#         self.low_y_cut = low_y_cut
#         self.high_y_cut = high_y_cut
        self.model=LinearRegression(n_jobs=n_jobs)
        
    def fit(self, x_train, y_train, low_y_cut=-0.085, high_y_cut=0.075): #x_train=x_train.fillna(d_mean)
        d_mean= x_train.median(axis=0)
        y_is_above_cut = (y_train > high_y_cut)
        y_is_below_cut = (y_train < low_y_cut)
        y_is_within_cut = (~y_is_above_cut & ~y_is_below_cut)
        self.model.fit(np.array(x_train.loc[y_is_within_cut, 'technical_20'].values).\
                                      reshape(-1,1), y_train.loc[y_is_within_cut])

    def predict(self, test):
        return self.model.predict(test).clip(low_y_cut, high_y_cut)
    
    def score(self, x_train, y_train):
        return self.model.score(x_train, y_train)

In [24]:
lr = LR_tech_20(n_jobs=3)
param_grid = { 
    'low_y_cut': [-0.085, -0.08, -0.075, -0.07],
    'high_y_cut': [0.09, 0.085, 0.08, 0.075, 0.07]
}

start=time()
CV_lr = GridSearchCV(estimator=lr, param_grid=param_grid, cv= 5)
CV_lr.fit(o.train[col].fillna(d_mean), o.train['y'])
print CV_fr.best_params_
end=time()
print(end-start)

TypeError: Cannot clone object '<__main__.LR_tech_20 instance at 0x0000000015A37348>' (type <type 'instance'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.

In [ ]:
ymean_dict = dict(o.train.groupby(["id"])["y"].median())

In [ ]:
class Ensemble(object):
    def __init__(self, n_folds, stacker, base_models):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models
        self.S_train=[]
        self.S_test=[]
        
    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        folds = list(KFold(len(y), n_folds=self.n_folds, shuffle=True, random_state=2016))
        self.S_train = np.zeros((X.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):
            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                # y_holdout = y[test_idx]
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_holdout)[:]
                S_train[test_idx, i] = y_pred
        
    def fit_predict(self, T):       
        T = np.array(T)       
        self.S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((T.shape[0], len(folds)))
            for j, (train_idx, test_idx) in enumerate(folds):
                S_test_i[:, j] = clf.predict(T)[:]
            self.S_test[:, i] = S_test_i.mean(1)
        self.stacker.fit(S_train, y)
        y_pred = self.stacker.predict(S_test)[:]
        return y_pred

In [43]:
start=timeit.timeit()

while True:
    test = o.features[col]
    n = test.isnull().sum(axis=1)
    for c in test.columns:
        test[c + '_nan_'] = pd.isnull(test[c])
    test = test.fillna(d_mean)
    test['znull'] = n
    pred = o.target
    test2 = np.array\
    (o.features[col].fillna(d_mean)['technical_20'].values).reshape(-1,1)
    pred['y'] = (model1.predict(test).clip(low_y_cut, high_y_cut) * 0.65)+\
    (model2.predict(test2).clip(low_y_cut, high_y_cut) * 0.35)
    pred['y'] = pred.apply(lambda r: 0.95 * r['y'] + \
                           0.05 * ymean_dict[r['id']] \
                           if r['id'] in ymean_dict else r['y'], axis = 1)
    pred['y'] = [float(format(x, '.6f')) for x in pred['y']]
    o, reward, done, info = env.step(pred)
    if done:
        print("el fin ...", info["public_score"])
        break
    if o.features.timestamp[0] % 100 == 0:
        print(reward)
        
end = timeit.timeit()
print(end - start)

/Users/jingguo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


-0.317732370655
-0.182276665545


KeyboardInterrupt: 

In [20]:
def fitInitialScript(low_y_cut=-0.075, high_y_cut=0.075, coeff1=0.65,\
                     coeff2=0.95):
    env = kagglegym.make()
    o = env.reset()
    
    y_is_above_cut = (o.train.y > high_y_cut)
    y_is_below_cut = (o.train.y < low_y_cut)
    y_is_within_cut = (~y_is_above_cut & ~y_is_below_cut)
    model2 = LinearRegression(n_jobs=-1)
    model2.fit(np.array(o.train[col].fillna(d_mean).\
                        loc[y_is_within_cut, 'technical_20'].values)\
               .reshape(-1,1), o.train.loc[y_is_within_cut, 'y'])
    #https://www.kaggle.com/ymcdull/two-sigma-financial-modeling/ridge-lb-0-0100659
    ymean_dict = dict(o.train.groupby(["id"])["y"].median())
    
    while True:
        test = o.features[col]
        n = test.isnull().sum(axis=1)
        for c in test.columns:
            test[c + '_nan_'] = pd.isnull(test[c])
        test = test.fillna(d_mean)
        test['znull'] = n
        pred = o.target
        test2 = np.array\
        (o.features[col].fillna(d_mean)['technical_20'].values).reshape(-1,1)
        pred['y'] = (model1.predict(test).clip(low_y_cut, high_y_cut) * coeff1)+\
        (model2.predict(test2).clip(low_y_cut, high_y_cut) * (1-coeff1))
        pred['y'] = pred.apply(lambda r: coeff2 * r['y'] + \
                           (1-coeff2) * ymean_dict[r['id']] \
                           if r['id'] in ymean_dict else r['y'], axis = 1)
        pred['y'] = [float(format(x, '.6f')) for x in pred['y']]
        o, reward, done, info = env.step(pred)
        if done:
            print("low=",low_y_cut, "high=", high_y_cut, "coeff1=",\
                  coeff1, "coeff2=", coeff2, \
                  "r=",info["public_score"])
            break
#         if o.features.timestamp[0] % 100 == 0:
#             print(reward)

In [47]:
fitInitialScript(-0.086093,0.093497)

/Users/jingguo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


el fin ... 0.0212274000456


In [49]:
n=10
x=-.075+np.arange(-n,n+1)/(n)*.01
y=.075+np.arange(-n,n+1)/(n)*.01
w= [.65,.7]
for i in [(a,b) for a in x for b in y for c in w]:
    fitInitialScript(*i)

/Users/jingguo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


low= -0.085 high= 0.065 r= 0.0209034018658
low= -0.085 high= 0.07 r= 0.0213959546743
low= -0.085 high= 0.075 r= 0.0216419710708
low= -0.085 high= 0.08 r= 0.0216557887025
low= -0.085 high= 0.085 r= 0.0215369685407
low= -0.08 high= 0.065 r= 0.0212120087397
low= -0.08 high= 0.07 r= 0.0215379611079
low= -0.08 high= 0.075 r= 0.0216506500503
low= -0.08 high= 0.08 r= 0.0215438737015
low= -0.08 high= 0.085 r= 0.0213106869652
low= -0.075 high= 0.065 r= 0.0214506190502
low= -0.075 high= 0.07 r= 0.0215877595609
low= -0.075 high= 0.075 r= 0.0215442626819
low= -0.075 high= 0.08 r= 0.0212951988375
low= -0.075 high= 0.085 r= 0.020927671549
low= -0.07 high= 0.065 r= 0.021580447962
low= -0.07 high= 0.07 r= 0.0215096098464
low= -0.07 high= 0.075 r= 0.0212877028251
low= -0.07 high= 0.08 r= 0.0208742867317
low= -0.07 high= 0.085 r= 0.0203466129444
low= -0.065 high= 0.065 r= 0.0215203751963
low= -0.065 high= 0.07 r= 0.02120781505
low= -0.065 high= 0.075 r= 0.0207703824355
low= -0.065 high= 0.08 r= 0.020152